In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

In [2]:
#loading data
data = pd.read_csv("NSECEWorkforceData.csv")

In [3]:
#data = data[data["WF9_CHAR_HEALTH_INSRNCE"] != -1]

In [4]:
#limiting data to where Y is defined
y = data["WF9_CESD7_TOT"]
valid = y >= 0

data = data[valid]
X = data.drop(["CB9_METH_CASEID", "WF9_COMPLETE_DATE", "WF9_METH_CASEID", "WF9_METH_FILEVERSION","WF9_MODE", "CB9_AGECAT_SERVE_1", "CB9_AGECAT_SERVE_0", "CB9_AGECAT_SERVE_2", "CB9_AGECAT_SERVE_3", "CB9_AGECAT_SERVE_4", "CB9_AGECAT_SERVE_5", "WF9_A14_FUFILL_REQ", "WF9_A17_CULTTRAIN", "WF9_ATTITUDES_PMS_PROG_IMP", "WF9_ATTITUDES_PMS_TOT_TRAD_IMP", "WF9_ATTITUDES_PMS_TRAD", "WF9_ATTITUDES_PMS_TRAD_IMP", "WF9_D1_OBEY", "WF9_D1_RIGHT", 
               "WF9_D1_OBEDIENCE", "WF9_D1_IDEAS", "WF9_D1_POV", "WF9_D1_DISAGREE", "WF9_D1_BEBAD", "WF9_D1_OBEYTEACH", "WF9_D1_DISPARENTS", "WF9_D1_PRETENDING", "WF9_G_ACTIVITY_PK_A", "WF9_G_ACTIVITY_PK_B", "WF9_G_ACTIVITY_PK_C", "WF9_G_ACTIVITY_PK_D", "WF9_G_ACTIVITY_PK_E", "WF9_G_ACTIVITY_PK_F", "WF9_G_ACTIVITY_PK_G", "WF9_G_ACTIVITY_PK_I", "WF9_D11_DEPRESSED", "WF9_D11_EAT", "WF9_D11_EFFORT", 
               "WF9_D11_GETGOING", "WF9_D11_RESTLESS", "WF9_D11_SAD", "WF9_D12_PAINT", "WF9_D13_CRY", "WF9_D14_HIT", "WF9_D15_PUZZLE", "WF9_G_ACTIVITY_IT_A", "WF9_G_ACTIVITY_IT_B", "WF9_G_ACTIVITY_IT_C", "WF9_G_ACTIVITY_IT_D", "WF9_G_ACTIVITY_IT_E", "WF9_G_ACTIVITY_IT_F", "WF9_G_ACTIVITY_IT_G", "WF9_G_ACTIVITY_IT_I", "WF9_METH_VSTRATUMPU", "WF9_METH_VPSUPU", "WF9_WEIGHT_CLSM", 
               "WF9_METH_WEIGHT", "WF9_PROFDEV_HELP_COST", "WF9_PROFDEV_HELP_TIME", "WF9_PROFDEV_HELP_TUITION", "WF9_WORK_WAGE_IMP", "WF9_CESD7_TOT", "WF9_CESD7_IMP_FLAG", "WF9_CESD7_TOT", "WF9_CESD7_CUT", "WF9_CESD7_IMP_FLAG", "WF9_D11_MIND", "CB9_SERVE_0TO5YRS", "WF9_E20_SRHEALTH", "WF9_WORK_CH_STRESS", "WF9_CAREER_SEARCH", "WF9_METH_DATA_SOURCE", "Unnamed: 0", "WF9_A13_COURSEOBS", "WF9_A20_DEMSKILLS_OBS", "WF9_C1_MOSTOFTEN",
               "WF9_CAREER_CERT_CDA", "WF9_CAREER_CERT_ECE"], axis = 1)
Y = data["WF9_CESD7_CUT"]


WageValid = X["WF9_WORK_WAGE"] > 0

X = X[WageValid]
Y = Y[WageValid]

In [5]:
X["WF9_QUEXLANG"] = X["WF9_QUEXLANG"].astype("float")

X_temp = X.copy()
for i in ["WF9_ATTITUDES_PMS_TOT_TRAD", "WF9_ATTITUDES_PMS_PROG"]:
    X = X[X_temp[i] != " "]
    Y = Y[X_temp[i] != " "]
    X_temp = X.copy()

In [6]:
for i in ["WF9_ATTITUDES_PMS_TOT_TRAD", "WF9_ATTITUDES_PMS_PROG"]:
    X[i] = X[i].astype("float")

In [7]:
sum(Y==1)/sum(Y==0)

0.09119496855345911

# Searching for statistically sign. variables

In [8]:
X = sm.add_constant(X)

model = sm.Logit(Y, X).fit()

Optimization terminated successfully.
         Current function value: 0.232347
         Iterations 9


sign = ["", "", "", "", "", "", "",
       "WF9_CAREER_PROFASSOC", "WF9_CHAR_FIRSTYRUS", "WF9_CHAR_COUNTRY_BORN", "WF9_CL2_A_ANY_STAFF_BLACK", "WF9_CL6A_B_PRCNT_CHCLASS_BLACK", "WF9_PROFDEV_TOPIC_NONHS", "WF9_PROFDEV_WRKSHP_TYPE", "WF9_QUEXLANG",
       "WF9_WORK_BGCHK_EASY", "WF9_WORK_CRCLM", "WF9_WORK_CLASSES", "WF9_WORK_HELP_AVAILABLE", "WF9_WORK_PLAN", "WF9_WORK_PLAN_WHEN", "WF9_WORK_PLAN_WHEN",
       "WF9_WORK_PRNTS_APPRCTE", "WF9_WORK_PRNTS_TALK", "WF9_WORK_RESPECT", "WF9_WORK_REVIEW", "WF9_WORK_YRS"]

In [9]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          WF9_CESD7_CUT   No. Observations:                 4164
Model:                          Logit   Df Residuals:                     4060
Method:                           MLE   Df Model:                          103
Date:                Sat, 22 Apr 2023   Pseudo R-squ.:                  0.1916
Time:                        01:40:18   Log-Likelihood:                -967.49
converged:                       True   LL-Null:                       -1196.8
Covariance Type:            nonrobust   LLR p-value:                 2.466e-46
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                             -7.9605      2.664     -2.989      0.003     -13.181      -2.740
CB9_AGECAT_SERVE_SA                0.0221      0.137      0.162      0.872      -0.246       0.290
CB9_PRGM_AUSPICE_EXP              -0.0194      0.049     -0.399      0.690      -0.115       0.076
CB9_RVNU_CENTER_FUND_COMBO        -0.0480      0.025     -1.959      0.050      -0.096     3.2e-05
CB9_RVNU_CHRG                     -0.1415      0.159     -0.891      0.373      -0.453       0.170
CB9_RVNU_CNTRBTNS                 -0.0133      0.114     -0.117      0.907      -0.237       0.210
CB9_RVNU_GOVT_FEDERAL             -0.0172      0.079     -0.217      0.828      -0.173       0.138
CB9_RVNU_GOVT_LOCAL               -0.0657      0.077     -0.856      0.392      -0.216       0.085
CB9_RVNU_GOVT_STATE                0.0903      0.113      0.797      0.425      -0.132       0.312
CB9_RVNU_PUBLIC_PRIVATE_MIX       -0.0175      0.043     -0.410      0.682      -0.101       0.066
CB9_SERVE_0TO3YRS                 -0.3710      0.197     -1.881      0.060      -0.757       0.016
CB9_SERVE_3TO5YRS                 -0.3636      0.242     -1.505      0.132      -0.837       0.110
WF9_A12_COLLENROLL                -0.0770      0.106     -0.726      0.468      -0.285       0.131
WF9_A18_HELP_SUP                  -0.0665      0.126     -0.525      0.599      -0.314       0.181
WF9_A19_HRS_SKILLS                -0.0818      0.049     -1.656      0.098      -0.179       0.015
WF9_ATTITUDES_PMS_PROG             0.0976      0.035      2.772      0.006       0.029       0.167
WF9_ATTITUDES_PMS_TOT_TRAD         0.0004      0.020      0.023      0.982      -0.038       0.039
WF9_C5_OVER4HR_TRAIN               0.2002      0.120      1.664      0.096      -0.036       0.436
WF9_CAREER_CERT                    0.0238      0.061      0.393      0.695      -0.095       0.142
WF9_CAREER_EXPERIENCE             -0.0269      0.050     -0.541      0.588      -0.124       0.070
WF9_CAREER_HBPAID                 -0.1555      0.122     -1.273      0.203      -0.395       0.084
WF9_CAREER_PROFASSOC               0.2221      0.143      1.550      0.121      -0.059       0.503
WF9_CAREER_REASON                  0.0007      0.035      0.020      0.984      -0.067       0.069
WF9_CAREER_SEARCH_WHY              0.0250      0.016      1.531      0.126      -0.007       0.057
WF9_CAREER_UNION                  -0.1116      0.190     -0.586      0.558      -0.485       0.262
WF9_CB_COMM_POVERTY_DENSITY        0.0797      0.081      0.985      0.325      -0.079       0.238
WF9_CB_COMM_URBAN_DENSITY         -0.0199      0.140     -0.143      0.886      -0.293       0.254
WF9_CHAR_CH_6TO12                 -0.0402      0.089     -0.451      0.652      -0.215       0.134
WF9_CHAR_CH_UNDER5                 0.0398      0.091      0.438      0.661      -0.138       0.218
WF9_CHAR_COUNTRY_BORN             -0.2292      0.143     -1.603      0.10

# Preparing Features

In [10]:
for column in ["WF9_CHAR_RACE", "WF9_CAREER_UNION", "WF9_CHAR_HISP",
                                                            "WF9_CHAR_COUNTRY_BORN", "WF9_CHAR_MARITAL", "WF9_CHAR_GOVT_PRGM",
                                                            "WF9_CHAR_GENDER", "WF9_C5_OVER4HR_TRAIN", "CB9_SERVE_0TO3YRS",
                                                            "WF9_PROFDEV_WRKSHP", "WF9_WORK_PLAN", "WF9_WORK_REVIEW", "CB9_RVNU_CENTER_FUND_COMBO",
               "WF9_CHAR_HEALTH_INSRNCE"
                                                           ]:
    c = data[column]
    c = c.replace(-1, 999)
    X[column] = c

In [11]:
#Making Dummies
columns = ["WF9_WORK_YRS", "WF9_CAREER_EXPERIENCE", "WF9_CHAR_EDUC", "WF9_A19_HRS_SKILLS",
           "WF9_WORK_HRS_CAT", "WF9_WORK_MONTHS",  "WF9_WORK_BEHAVIOR", "WF9_WORK_HELP_AVAILABLE",
           "WF9_WORK_RESPECT", "WF9_CHAR_HHINCOME", "WF9_CHAR_RACE",
          "WF9_CAREER_UNION", "WF9_CHAR_HISP", "WF9_CHAR_COUNTRY_BORN", "WF9_CHAR_MARITAL",
          "WF9_CHAR_GENDER", "WF9_CHAR_GOVT_PRGM", "WF9_WORK_PRNTS_TALK",
          "WF9_C5_OVER4HR_TRAIN", "WF9_ATTITUDES_PMS_PROG", "CB9_SERVE_0TO3YRS", "CB9_RVNU_CENTER_FUND_COMBO",
          "WF9_CL6A_B_PRCNT_CHCLASS_BLACK", "WF9_PROFDEV_WRKSHP", "WF9_WORK_BGCHK_EASY", "WF9_WORK_PLAN",
          "WF9_WORK_REVIEW", "WF9_CHAR_HEALTH_INSRNCE"]

Output = X[columns]

before = Output.copy()
before["WF9_CESD7_CUT"] = Y
before.to_csv("beforedummies.csv")






#Refuse to Answer Columns
variables_r =  ["WF9_WORK_YRS", "WF9_CAREER_EXPERIENCE", "WF9_CHAR_EDUC", "WF9_A19_HRS_SKILLS", 
                "WF9_WORK_HRS_CAT", "WF9_WORK_MONTHS", "WF9_WORK_BEHAVIOR", "WF9_WORK_HELP_AVAILABLE",
                "WF9_WORK_RESPECT", "WF9_CHAR_HHINCOME", "WF9_CL6A_B_PRCNT_CHCLASS_BLACK",
               "WF9_WORK_BGCHK_EASY"]
for var in variables_r:
    Output[var + "_999"] = [int(x) for x in Output[var] == -1]


Output = pd.get_dummies(Output, drop_first = True, columns=["WF9_CHAR_RACE", "WF9_CAREER_UNION", "WF9_CHAR_HISP",
                                                            "WF9_CHAR_COUNTRY_BORN", "WF9_CHAR_MARITAL", "WF9_CHAR_GOVT_PRGM",
                                                            "WF9_CHAR_GENDER", "WF9_C5_OVER4HR_TRAIN", "CB9_SERVE_0TO3YRS",
                                                            "WF9_PROFDEV_WRKSHP", "WF9_WORK_PLAN", "WF9_WORK_REVIEW", "CB9_RVNU_CENTER_FUND_COMBO",
                                                            "WF9_CHAR_HEALTH_INSRNCE"
                                                           ])


#Output["WF9_CL6A_B_PRCNT_CHCLASS_BLACK" + "_AL1"] = [int(x) for x in Output[var] == -2]
Output = Output

/tmp/ipykernel_240/2212402391.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Output[var + "_999"] = [int(x) for x in Output[var] == -1]


In [12]:
train_X = sm.add_constant(Output)
model = sm.Logit(Y, train_X).fit()

         Current function value: 0.237181
         Iterations: 35


/opt/conda/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [13]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          WF9_CESD7_CUT   No. Observations:                 4164
Model:                          Logit   Df Residuals:                     4092
Method:                           MLE   Df Model:                           71
Date:                Sat, 22 Apr 2023   Pseudo R-squ.:                  0.1748
Time:                        01:40:18   Log-Likelihood:                -987.62
converged:                      False   LL-Null:                       -1196.8
Covariance Type:            nonrobust   LLR p-value:                 1.140e-50
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -6.7682      0.882     -7.671      0.000      -8.497      -5.039
WF9_WORK_YRS                          -0.0514      0.019     -2.767      0.006      -0.088      -0.015
WF9_CAREER_EXPERIENCE                  0.0062      0.048      0.129      0.897      -0.088       0.100
WF9_CHAR_EDUC                         -0.0987      0.049     -1.999      0.046      -0.195      -0.002
WF9_A19_HRS_SKILLS                    -0.0564      0.050     -1.132      0.258      -0.154       0.041
WF9_WORK_HRS_CAT                       0.0786      0.045      1.749      0.080      -0.009       0.167
WF9_WORK_MONTHS                        0.0101      0.022      0.451      0.652      -0.034       0.054
WF9_WORK_BEHAVIOR                      0.5607      0.093      6.036      0.000       0.379       0.743
WF9_WORK_HELP_AVAILABLE                0.1567      0.064      2.434      0.015       0.031       0.283
WF9_WORK_RESPECT                       0.5052      0.063      8.079      0.000       0.383       0.628
WF9_CHAR_HHINCOME                     -0.0398      0.055     -0.729      0.466      -0.147       0.067
WF9_WORK_PRNTS_TALK                    0.2831      0.089      3.170      0.002       0.108       0.458
WF9_ATTITUDES_PMS_PROG                 0.1089      0.025      4.276      0.000       0.059       0.159
WF9_CL6A_B_PRCNT_CHCLASS_BLACK        -0.0051      0.003     -1.956      0.050      -0.010    1.04e-05
WF9_WORK_BGCHK_EASY                    0.1411      0.074      1.918      0.055      -0.003       0.285
WF9_WORK_YRS_999                     -22.1199   9.34e+04     -0.000      1.000   -1.83e+05    1.83e+05
WF9_CAREER_EXPERIENCE_999              1.5549      0.570      2.727      0.006       0.437       2.673
WF9_CHAR_EDUC_999                    -20.5517   2.41e+04     -0.001      0.999   -4.73e+04    4.73e+04
WF9_A19_HRS_SKILLS_999                -0.0560      0.686     -0.082      0.935      -1.400       1.288
WF9_WORK_HRS_CAT_999                   1.5392      1.135      1.356      0.175      -0.685       3.763
WF9_WORK_MONTHS_999                   -0.4272      0.744     -0.574      0.566      -1.886       1.031
WF9_WORK_BEHAVIOR_999                  2.0875      0.689      3.031      0.002       0.737       3.438
WF9_WORK_HELP_AVAILABLE_999          -17.2985   4575.755     -0.004      0.997   -8985.614    8951.016
WF9_WORK_RESPECT_999                   1.9872      1.137      1.748      0.080      -0.241       4.216
WF9_CHAR_HHINCOME_999                 -0.4112      0.340     -1.210      0.226      -1.078       0.255
WF9_CL6A_B_PRCNT_CHCLASS_BLACK_999    -0.1425      0.170     -0.838      0.402      -0.476       0.191
WF9_WORK_BGCHK_EASY_999                0.2715      0.487      0.558      0.577      -0.683       1.225
WF9_CHAR_RACE_2                        0.0847      0.186      0.454      0.650      -0.281       0.450
WF9_CHAR_RACE_3                        0.3404   

# Finalizing Tables

In [14]:
Output_log = Output.copy()

#Adding Y to tables so entire table is together
Output_log["WF9_CESD7_CUT"] = Y

# Exporting

In [15]:
Output_log.to_csv("Cleaning_Output_Log", index = False)